In [1]:
import pandas as pd
import numpy as np

# Graveyard

## Creating DataFrames

### First Try
```python
excel = pd.read_excel('steam tables.xlsx',sheet_name='Saturated Water (Temp Table)')
df = pd.DataFrame()
for column in excel.columns:
    df[column] = excel[column].apply(lambda x: x.split()).explode().reset_index().drop('index',axis = 1).squeeze()
df = df.set_index(df.columns[0])
for columns in df.columns:
    df[columns] = df[columns].apply(lambda x: float(x))
df['Sat Liquid vf'] = df['Sat Liquid vf'].apply(lambda x: x/1000)
df```

### Second Try
```python
def createDataFrame(sheet):
    if sheet == 'Saturated Water (Temp Table)':
        global SatWatTemptbl
        SatWatTemptbl = pd.DataFrame()
        print(1)
        return SatWatTemptbl
    elif sheet == 'Saturated Water (Pres Table)':
        global SatWatPrestbl
        SatWatPrestbl = pd.DataFrame()
        print(2)
        return SatWatPrestbl
        
def finishDataFrame(df):
    for column in excel.columns:
        df[column] = excel[column].apply(lambda x: x.split()).explode().reset_index().drop('index',axis = 1).squeeze()
    for columns in df.columns:
        df[columns] = df[columns].apply(lambda x: float(x))
    df = df.set_index(df.columns[0])

sheetNames = ['Saturated Water (Temp Table)','Saturated Water (Pres Table)']

for sheet in sheetNames:
    excel = pd.read_excel('steam tables.xlsx',sheet_name=sheet)
    finishDataFrame(createDataFrame(sheet))```
#df['Sat Liquid vf'] = df['Sat Liquid vf'].apply(lambda x: x/1000)
```

### Third Try
##### Looping didnt work as expected, only outputs the last table

```python
sheetNames = ['Saturated Water (Temp Table)','Saturated Water (Pres Table)']#,'Superheated Water']

class thermoTable:
    def __init__(self,sheetnameslist):
        #for sheet in sheetnameslist:
            #self.excel = pd.read_excel('steam tables.xlsx', sheet_names=sheet)
            self.sheetnameslist = sheetnameslist
            if sheet == 'Saturated Water (Temp Table)' in sheetnameslist:
                global SatWatTemptbl
                SatWatTemptbl = pd.DataFrame()
            elif sheet == 'Saturated Water (Pres Table)' in sheetnameslist:
                global SatWatPrestbl
                SatWatPrestbl = pd.DataFrame()          
            elif sheet == 'Superheated Water':
                global SupWattbl
                SupWattbl = pd.DataFrame()
    def outputdf(self,df):
        for sheet in self.sheetnameslist:
            self.excel = pd.read_excel('steam tables.xlsx', sheet_names=sheet)
            for column in self.excel.columns:
                df[column] = self.excel[column].apply(lambda x: x.split()).explode().reset_index().drop('index',axis = 1).squeeze()
            for columns in df.columns:
                df[columns] = df[columns].apply(lambda x: float(x))
            df = df.set_index(df.columns[0])
            if 'Sat Liquid vf' in df.columns:
                df['Sat Liquid vf'] = df['Sat Liquid vf'].apply(lambda x: x/1000)
        return df
tT = thermoTable(sheetNames)
tT.outputdf(SatWatPrestbl)
tT.outputdf(SatWatTemptbl)
tT.outputdf(SupWattbl)
```

### Superheated Calcs
#### First Try: Elements that were scrapped

##### Took out trytemperature
```python
def trytemperature(p, t):
    '''Check to see if the given temperature is inside the range of the specified pressure'''
    if t <= SupWat.xs(p, level=0).T.keys().max() and t >= SupWat.xs(p, level=0).T.keys().min():
        return True
    else:
        return 'Temperature not in range'```

##### Took out trypressure
```python
def trypressure(p):
    try: 
        SupWat.xs(p, level=0)
        return True
    except KeyError: 
        return 'Pressure not found'```
        
##### Took out anything to do with trypressure
```python
def flowtointerpolatefrompressure(p, z, column):
    took off trypressure ifelse for Superheated2
    if trypressure(p) is True:
        if z > SupWat.xs(p, level=0)[column].values[-1]:
            return f'Inputed value ({z}) too large'
        elif z < SupWat.xs(p, level=0)[column].values[0]:
            return f'Inputed value ({z}) too small'
        elif z == SupWat.xs(p, level=0)[column].values[-1]:
            return SupWat.xs(p, level=0)[column].keys()[-1]
        elif z == SupWat.xs(p, level=0)[column].values[0]:
            return SupWat.xs(p, level=0)[column].keys()[0]
        else:
            return interpolatetempfrompressure(p, z, column)
    else:
        return trypressure(p)```
        
##### Remade the flow for Superheated
```python        
def Superheated(p=None, t=None, v=None, u=None, h=None, s=None):
    
    if list(np.isin([p, t, v, u, h, s], None)).count(True) != 4:
        return print('Enter exactly 2 values')
    
    elif p is not None and t is not None:
        if trypressure(p) is True and trytemperature(p, t) is True:
            if t in SupWat.xs(p, level=0).T:
                return print(dict(SupWat.xs((p, t))))
            else: 
                return print(interpolatetemp(p, t))
        #elif trypressure(p) is not True and trytemperature(p, t) is True:
            #return print(trypressure(p)) # wont happen because p is not true
        #elif trypressure(p) is True and trytemperature(p, t) is not True:
            #return print('bruh it aint workin')
        #else: 
            #return print(trypressure(p))
            
    elif p is not None and v is not None:
        return print(prettyreturn(p, v, 'v'))
       
    elif p is not None and u is not None:
        return print(prettyreturn(p, u, 'u'))
        
    elif p is not None and h is not None:
        return print(prettyreturn(p, h, 'h'))
        
    elif p is not None and s is not None:
        return print(prettyreturn(p, s, 's'))```

#### Second Try:
##### Worked great but made it instantiable for Compressed water table

```python
pressure_list = np.unique(SupWat.index.get_level_values(0))


def where_pressure(p):
    """Determines whether pressure is on table."""
    if p in pressure_list:
        return pressure_list.tolist().index(p)  # returns index (int) of pressure table
    elif p < pressure_list[0] or p > pressure_list[-1]:
        return 'Pressure not in range'  # pressure not in pressure table
    elif pressure_list[0] < p < pressure_list[-1]:
        pres_position = np.searchsorted(pressure_list, p)
        return pres_position  # returns where pressure is between table


def interpolation_instruction(p):
    """Gives instructions on what path to follow based on where pressure is on table."""
    if type(where_pressure(p)) is int:
        return 'on table'
    elif type(where_pressure(p)) is np.int64:
        return 'between table'
    else:
        return where_pressure(p)


def interpolate_temp(p, t):
    """Simple interpolation given a pressure on the table and a random temperature.
        Returns dictionary with all elements from table."""
    temp_df = SupWat.xs(p, level=0).T
    position = np.searchsorted(temp_df.keys(), t)
    x = (t - temp_df.keys()[position - 1]) / (temp_df.keys()[position] - temp_df.keys()[position - 1])
    list_df = SupWat.xs(p, level=0)
    dictionary = {}
    for names in SupWat.xs(p, level=0).columns:
        y = x * (list_df[names].values[position] - list_df[names].values[position - 1]) + list_df[names].values[
            position - 1]
        dictionary.update({names: y})
    return dictionary


def interpolate_temp_from_pressure(p, z, column):
    """Simple interpolation given a pressure on the table and a random element.
        Returns temperature for the corresponding element."""
    list_df = SupWat.xs(p, level=0)[column]
    position = np.searchsorted(list_df.values, z)
    x = (z - list_df.values[position - 1]) / (list_df.values[position] - list_df.values[position - 1])
    y = x * (list_df.keys()[position] - list_df.keys()[position - 1]) + list_df.keys()[position - 1]
    return y


def flow_to_interpolate_from_pressure(p, z, column):
    """Gives instructions for how to execute code given a pressure on the table and a random element."""
    if z > SupWat.xs(p, level=0)[column].values[-1]:
        return f'Inputted value ({z}) too large'
    elif z < SupWat.xs(p, level=0)[column].values[0]:
        return f'Inputted value ({z}) too small'
    elif z == SupWat.xs(p, level=0)[column].values[-1]:
        return SupWat.xs(p, level=0)[column].keys()[-1]
    elif z == SupWat.xs(p, level=0)[column].values[0]:
        return SupWat.xs(p, level=0)[column].keys()[0]
    else:
        return interpolate_temp_from_pressure(p, z, column)


def pretty_return(p, z, column):
    """Code wrapper for efficient output."""
    output = flow_to_interpolate_from_pressure(p, z, column)
    if type(output) is np.float64:
        return [output, interpolate_temp(p, output)]
    else:
        return output


def output_final_dict(dh, dl, xp):
    """Returns final interpolated values for each element in table using pressure position."""
    fd = {}
    for k in dh:
        z = xp * (dh.get(k) - dl.get(k)) + dl.get(k)
        fd.update({k: z})
    return fd


def triple_interpolation_variables(p):
    """Establishes variables used in triple interpolation."""
    pres_position = np.searchsorted(pressure_list, p)
    x_pres = (p - pressure_list[pres_position - 1]) / (pressure_list[pres_position] - pressure_list[pres_position - 1])
    temp_df_high = SupWat.xs(pressure_list[pres_position], level=0).T
    temp_df_low = SupWat.xs(pressure_list[pres_position - 1], level=0).T
    return [pres_position, x_pres, temp_df_high, temp_df_low]


def triple_interpolation_t_element(section, pres_position, t):
    """Element of triple interpolation when temperature is given.
        Returns dictionary with each interpolated element using temperature position."""
    column_names = SupWat.xs(pressure_list[pres_position], level=0).columns
    interp_dictionary = {}
    t_pos = np.searchsorted(section.keys(), t)
    x = (t - section.keys()[t_pos - 1]) / (section.keys()[t_pos] - section.keys()[t_pos - 1])
    for names in column_names:
        y = x * (section.T[names].values[t_pos] - section.T[names].values[t_pos - 1]) + section.T[names].values[
            t_pos - 1]
        interp_dictionary.update({names: y})
    return interp_dictionary


def triple_interpolation_t(p, t):
    """Triple interpolation given temperature flow code.
        Returns dictionary of properly interpolated elements."""
    [pres_position, x_pres, temp_df_high, temp_df_low] = triple_interpolation_variables(p)

    if temp_df_high.keys()[0] < t < temp_df_high.keys()[-1] and temp_df_low.keys()[0] < t < temp_df_low.keys()[-1]:
        if t in temp_df_high.keys() and t in temp_df_low.keys():
            d_high = temp_df_high[t].to_dict()
            d_low = temp_df_low[t].to_dict()
            return output_final_dict(d_high, d_low, x_pres)

        elif t in temp_df_high.keys() and t not in temp_df_low.keys():
            d_high = temp_df_high[t].to_dict()
            d_low = triple_interpolation_t_element(temp_df_low, pres_position, t)
            return output_final_dict(d_high, d_low, x_pres)

        elif t not in temp_df_high.keys() and t in temp_df_low.keys():
            d_high = triple_interpolation_t_element(temp_df_high, pres_position, t)
            d_low = temp_df_low[t].to_dict()
            return output_final_dict(d_high, d_low, x_pres)

        else:
            d_high = triple_interpolation_t_element(temp_df_high, pres_position, t)
            d_low = triple_interpolation_t_element(temp_df_low, pres_position, t)
            return output_final_dict(d_high, d_low, x_pres)

    elif t < temp_df_high.keys()[0] or t < temp_df_low.keys()[0]:
        return f'Temperature must be higher than {max([temp_df_high.keys()[0], temp_df_low.keys()[0]])} C'

    elif t > temp_df_high.keys()[-1] or t > temp_df_low.keys()[-1]:
        return f'Temperature must be lower than {min([temp_df_high.keys()[-1], temp_df_low.keys()[-1]])} C'

    elif t == max([temp_df_high.keys()[0], temp_df_low.keys()[0]]):
        max_t_pos = np.argmax([temp_df_high.keys()[0], temp_df_low.keys()[0]])
        if max_t_pos == 0:
            d_high = temp_df_high[t].to_dict()
            d_low = triple_interpolation_t_element(temp_df_low, pres_position, t)
            return output_final_dict(d_high, d_low, x_pres)

        elif max_t_pos == 1:
            d_high = triple_interpolation_t_element(temp_df_high, pres_position, t)
            d_low = temp_df_low[t].to_dict()
            return output_final_dict(d_high, d_low, x_pres)

    elif t == min([temp_df_high.keys()[-1], temp_df_low.keys()[-1]]):
        min_t_pos = np.argmin([temp_df_high.keys()[-1], temp_df_low.keys()[-1]])
        if min_t_pos == 0:
            d_high = temp_df_high[t].to_dict()
            d_low = triple_interpolation_t_element(temp_df_low, pres_position, t)
            return output_final_dict(d_high, d_low, x_pres)

        elif min_t_pos == 1:
            d_high = triple_interpolation_t_element(temp_df_high, pres_position, t)
            d_low = temp_df_low[t].to_dict()
            return output_final_dict(d_high, d_low, x_pres)


def triple_interpolation_z_element(temp_df, zdf, z, pres_position, inout):
    """Element of triple interpolation when an elemental value is given.
        Depending on whether the elemental value appears on a table, it follows a flow where both instances
        return its corresponding temperature and
        dictionary with each interpolated element."""
    if inout == 'in':
        z_pos = np.where(zdf.values == z)[0][0]
        y = temp_df.keys()[z_pos]
        return [y, temp_df[y].to_dict()]
    elif inout == 'out':
        z_pos = np.searchsorted(zdf.values, z)
        x = (z - zdf.values[z_pos - 1]) / (zdf.values[z_pos] - zdf.values[z_pos - 1])
        y = x * (zdf.keys()[z_pos] - zdf.keys()[z_pos - 1]) + zdf.keys()[z_pos - 1]
        return [y, triple_interpolation_t_element(temp_df, pres_position, y)]


def triple_interpolation_z(p, z, column):
    """Triple interpolation given an elemental value; flow code.
        Returns list with interpolated temperature and dictionary of properly interpolated elements."""
    [pres_position, x_pres, temp_df_high, temp_df_low] = triple_interpolation_variables(p)
    zdf_high = SupWat.xs(pressure_list[pres_position], level=0)[column]
    zdf_low = SupWat.xs(pressure_list[pres_position - 1], level=0)[column]

    column_units = {'v': 'm3/kg', 'u': 'kJ/kg', 'h': 'kJ/kg', 's': 'kJ/kg-K'}

    if zdf_high.values[0] < z < zdf_high.values[-1] and zdf_low.values[0] < z < zdf_low.values[-1]:

        if z in zdf_high.values and z in zdf_low.values:
            [y_high, d_high] = triple_interpolation_z_element(temp_df_high, zdf_high, z, pres_position, 'in')
            [y_low, d_low] = triple_interpolation_z_element(temp_df_low, zdf_low, z, pres_position, 'in')
            final_t = x_pres * (y_high - y_low) + y_low
            return [final_t, output_final_dict(d_high, d_low, x_pres)]

        elif z in zdf_high.values and z not in zdf_low.values:
            [y_high, d_high] = triple_interpolation_z_element(temp_df_high, zdf_high, z, pres_position, 'in')
            [y_low, d_low] = triple_interpolation_z_element(temp_df_low, zdf_low, z, pres_position, 'out')
            final_t = x_pres * (y_high - y_low) + y_low
            return [final_t, output_final_dict(d_high, d_low, x_pres)]

        elif z not in zdf_high.values and z in zdf_low.values:
            [y_high, d_high] = triple_interpolation_z_element(temp_df_high, zdf_high, z, pres_position, 'out')
            [y_low, d_low] = triple_interpolation_z_element(temp_df_low, zdf_low, z, pres_position, 'in')
            final_t = x_pres * (y_high - y_low) + y_low
            return [final_t, output_final_dict(d_high, d_low, x_pres)]

        else:  # z not in either
            [y_high, d_high] = triple_interpolation_z_element(temp_df_high, zdf_high, z, pres_position, 'out')
            [y_low, d_low] = triple_interpolation_z_element(temp_df_low, zdf_low, z, pres_position, 'out')
            final_t = x_pres * (y_high - y_low) + y_low
            return [final_t, output_final_dict(d_high, d_low, x_pres)]

    elif z < zdf_high.values[0] or z < zdf_low.values[0]:
        return f'Inputted value ({column}) must be higher than {max([zdf_high.values[0], zdf_low.values[0]])} {column_units[column]}'

    elif z > zdf_high.values[-1] or z > zdf_low.values[-1]:
        return f'Inputted value ({column}) must be lower than {min([zdf_high.values[-1], zdf_low.values[-1]])} {column_units[column]}'

    elif z == max([zdf_high.values[0], zdf_low.values[0]]):
        max_z_pos = np.argmax([zdf_high.values[0], zdf_low.values[0]])
        if max_z_pos == 0:
            y_high = temp_df_high.keys()[0]
            d_high = temp_df_high[y_high].to_dict()
            [y_low, d_low] = triple_interpolation_z_element(temp_df_low, zdf_low, z, pres_position, 'out')
            final_t = x_pres * (y_high - y_low) + y_low
            return [final_t, output_final_dict(d_high, d_low, x_pres)]

        elif max_z_pos == 1:
            [y_high, d_high] = triple_interpolation_z_element(temp_df_high, zdf_high, z, pres_position, 'out')
            y_low = temp_df_low.keys()[0]
            d_low = temp_df_low[y_low].to_dict()
            final_t = x_pres * (y_high - y_low) + y_low
            return [final_t, output_final_dict(d_high, d_low, x_pres)]

    elif z == min([zdf_high.values[-1], zdf_low.values[-1]]):
        min_z_pos = np.argmin([zdf_high.values[-1], zdf_low.values[-1]])
        if min_z_pos == 0:
            y_high = temp_df_high.keys()[-1]
            d_high = temp_df_high[y_high].to_dict()
            [y_low, d_low] = triple_interpolation_z_element(temp_df_low, zdf_low, z, pres_position, 'out')
            final_t = x_pres * (y_high - y_low) + y_low
            return [final_t, output_final_dict(d_high, d_low, x_pres)]

        if min_z_pos == 1:
            [y_high, d_high] = triple_interpolation_z_element(temp_df_high, zdf_high, z, pres_position, 'out')
            y_low = temp_df_low.keys()[-1]
            d_low = temp_df_low[y_low].to_dict()
            final_t = x_pres * (y_high - y_low) + y_low
            return [final_t, output_final_dict(d_high, d_low, x_pres)]


def superheated(p=None, t=None, v=None, u=None, h=None, s=None):
    """Flow instructions for reading a superheated water vapor table."""
    if [p, t, v, u, h, s].count(None) != 4:
        return 'Enter exactly 2 values'

    elif p is None:  # maybe temporary?
        return 'Need a pressure'

    elif p is not None:
        if interpolation_instruction(p) == 'on table':
            if t is not None:
                if SupWat.xs(p, level=0).T.keys().max() >= t >= SupWat.xs(p, level=0).T.keys().min():
                    if t in SupWat.xs(p, level=0).T:
                        return dict(SupWat.xs((p, t)))
                    else:
                        return interpolate_temp(p, t)
                else:
                    return f'Temperature must be within the range of {SupWat.xs(p, level=0).T.keys().min()} - {SupWat.xs(p, level=0).T.keys().max()} C'

            elif v is not None:
                return pretty_return(p, v, 'v')

            elif u is not None:
                return pretty_return(p, u, 'u')

            elif h is not None:
                return pretty_return(p, h, 'h')

            elif s is not None:
                return pretty_return(p, s, 's')

        elif interpolation_instruction(p) == 'Pressure not in range':
            return interpolation_instruction(p)

        elif interpolation_instruction(p) == 'between table':
            if t is not None:
                return triple_interpolation_t(p, t)

            if v is not None:
                return triple_interpolation_z(p, v, 'v')

            if u is not None:
                return triple_interpolation_z(p, u, 'u')

            if h is not None:
                return triple_interpolation_z(p, h, 'h')

            if s is not None:
                return triple_interpolation_z(p, s, 's')


superheated(p=10, v=0.2)```

***
# Create DataFrames for each table
***

In [116]:
pd.ExcelFile('steam tables.xlsx').sheet_names

['Saturated Water (Temp Table)',
 'Saturated Water (Pres Table)',
 'Superheated Water',
 'Compressed Water',
 'Superheated Water Table 2']

In [107]:
SatWatTemp = pd.DataFrame()
SatWatPres = pd.DataFrame()
SatWat = pd.DataFrame()
SupWat = pd.DataFrame()
SupWat2 = pd.DataFrame()
CompWat = pd.DataFrame()

def out(df, sheet):
    excel = pd.read_excel('steam tables.xlsx', sheet_name=sheet)
    for column in excel.columns:
        df[column] = excel[column].apply(lambda x: x.split()).explode().reset_index().drop('index',axis = 1).squeeze()
    for column in df.columns:
        df[column] = df[column].apply(lambda x: float(x))
    df.set_index([df.columns[0],df.columns[1]],inplace=True)
    if 'Sat Liq vf' in df.columns:
        df['Sat Liq vf'] = df['Sat Liq vf'].div(1000)

sheetNames = ['Saturated Water (Temp Table)', 'Saturated Water (Pres Table)', 'Superheated Water', 'Compressed Water']
out(SatWatTemp, sheetNames[0])
out(SatWatPres, sheetNames[1])
out(SupWat, sheetNames[2])
out(CompWat, sheetNames[3])

SatWat = pd.concat([SatWatTemp.reset_index(), SatWatPres.reset_index()[['Temp', 'Pres', 'Sat Liq vf', 'Sat Vap vg', 'Sat Liq uf', 'Sat Vap ug', 'Sat Liq hf', 'Evap hfg', 'Sat Vap hg', 'Sat Liq sf', 'Sat Vap sg']]])
SatWat = SatWat.sort_values(by=['Temp']).drop_duplicates(subset=['Pres'])
SatWat = SatWat.set_index([SatWat.columns[0], SatWat.columns[1]])

CompWat['v'] = CompWat['v'].div(1000)

# table with larger range of pressures and temperatures but unsure of accuracy (especially at lower pressures)
def SupWatnewtable(SupWatT2):
    SupWatT2 = pd.read_excel('steam tables.xlsx', sheet_name = 'Superheated Water Table 2')
    SupWatT2['fill in P'] = SupWatT2['P, Mpa']
    SupWatT2.set_index(['P, Mpa', 'Temp C'], inplace=True)
    for name in SupWatT2.columns:
        SupWatT2[name] = SupWatT2[name].apply(lambda x: str(x).replace(' ', '')).apply(lambda x: float(x))
    SupWatT2['v'] = SupWatT2['v (m^3/Mg)'].div(1000)
    SupWatT2['u'] = SupWatT2['h'] - (SupWatT2['v (m^3/Mg)'] * SupWatT2['fill in P'])
    SupWatT2 = SupWatT2.drop(['v (m^3/Mg)', 'fill in P'], axis=1)[['v', 'u', 'h', 's']]
    SupWatT2.index = SupWatT2.index.set_levels(np.around(SupWatT2.index.levels[0]*10.0, decimals = 10), level=0)
    SupWatT2 = SupWatT2.rename_axis(index=['Press bar', 'Temp C'])
    return SupWatT2

SupWat2 = SupWatnewtable(SupWat2)

In [108]:
SatWat

,,Sat Liq vf,Sat Vap vg,Sat Liq uf,Sat Vap ug,Sat Liq hf,Evap hfg,Sat Vap hg,Sat Liq sf,Sat Vap sg
Temp,Pres,,,,,,,,,
0.01,0.00611,0.001000,206.136000,0.00,2375.3,0.01,2501.3,2501.4,0.0000,9.1562
4.00,0.00813,0.001000,157.232000,16.77,2380.9,16.78,2491.9,2508.7,0.0610,9.0514
5.00,0.00872,0.001000,147.120000,20.97,2382.3,20.98,2489.6,2510.6,0.0761,9.0257
6.00,0.00935,0.001000,137.734000,25.19,2383.6,25.20,2487.2,2512.4,0.0912,9.0003
8.00,0.01072,0.001000,120.917000,33.59,2386.4,33.60,2482.5,2516.1,0.1212,8.9501
...,...,...,...,...,...,...,...,...,...,...
357.10,180.00000,0.001840,0.007489,1698.90,2374.3,1732.00,777.1,2509.1,3.8715,5.1044
360.00,186.50000,0.001893,0.006945,1725.20,2351.5,1760.50,720.5,2481.0,3.9147,5.0526
361.50,190.00000,0.001924,0.006657,1739.90,2338.1,1776.50,688.0,2464.5,3.9388,5.0228


In [109]:
SatWatTemp

,,Sat Liq vf,Sat Vap vg,Sat Liq uf,Sat Vap ug,Sat Liq hf,Evap hfg,Sat Vap hg,Sat Liq sf,Sat Vap sg
Temp,Pres,,,,,,,,,
0.01,0.00611,0.001000,206.136000,0.00,2375.3,0.01,2501.3,2501.4,0.0000,9.1562
4.00,0.00813,0.001000,157.232000,16.77,2380.9,16.78,2491.9,2508.7,0.0610,9.0514
5.00,0.00872,0.001000,147.120000,20.97,2382.3,20.98,2489.6,2510.6,0.0761,9.0257
6.00,0.00935,0.001000,137.734000,25.19,2383.6,25.20,2487.2,2512.4,0.0912,9.0003
8.00,0.01072,0.001000,120.917000,33.59,2386.4,33.60,2482.5,2516.1,0.1212,8.9501
...,...,...,...,...,...,...,...,...,...,...
300.00,85.81000,0.001404,0.021670,1332.00,2563.0,1344.00,1404.9,2749.0,3.2534,5.7045
320.00,112.70000,0.001499,0.015490,1444.60,2525.5,1461.50,1238.6,2700.1,3.4480,5.5362
340.00,145.90000,0.001638,0.010800,1570.30,2464.6,1594.20,1027.9,2622.0,3.6594,5.3357


In [110]:
SatWatPres

,,Sat Liq vf,Sat Vap vg,Sat Liq uf,Sat Vap ug,Sat Liq hf,Evap hfg,Sat Vap hg,Sat Liq sf,Sat Vap sg
Pres,Temp,,,,,,,,,
0.04,28.96,0.001004,34.800000,121.45,2415.2,121.46,2432.9,2554.4,0.4226,8.4746
0.06,36.16,0.001006,23.739000,151.53,2425.0,151.53,2415.9,2567.4,0.5210,8.3304
0.08,41.51,0.001008,18.103000,173.87,2432.2,173.88,2403.1,2577.0,0.5926,8.2287
0.10,45.81,0.001010,14.674000,191.82,2437.9,191.83,2392.8,2584.7,0.6493,8.1502
0.20,60.06,0.001017,7.649000,251.38,2456.7,251.40,2358.3,2609.7,0.8320,7.9085
0.30,69.10,0.001022,5.229000,289.20,2468.4,289.23,2336.1,2625.3,0.9439,7.7686
0.40,75.87,0.001027,3.993000,317.53,2477.0,317.58,2319.2,2636.8,1.0259,7.6700
0.50,81.33,0.001030,3.240000,340.44,2483.9,340.49,2305.4,2645.9,1.0910,7.5939
0.60,85.94,0.001033,2.732000,359.79,2489.6,359.86,2293.6,2653.5,1.1453,7.5320


In [111]:
SupWat

v       u       h       s
Pres   Temp                                    
0.06   36.16   23.73900  2425.0  2567.4  8.3304
       80.00   27.13200  2487.3  2650.1  8.5804
       120.00  30.21900  2544.7  2726.0  8.7840
       160.00  33.30200  2602.7  2802.5  8.9693
       200.00  36.38300  2661.4  2879.7  9.1398
...                 ...     ...     ...     ...
320.00 640.00   0.01150  3184.5  3552.5  6.3290
       700.00   0.01273  3325.4  3732.8  6.5203
       740.00   0.01350  3415.9  3847.8  6.6361
       800.00   0.01460  3548.0  4015.1  6.7966
       900.00   0.01633  3762.7  4285.1  7.0372

[278 rows x 4 columns]

In [113]:
CompWat

v        u        h       s
Pres  Temp                                     
25.0  20.0   0.001001    83.80    86.30  0.2961
      40.0   0.001007   167.25   169.77  0.5715
      80.0   0.001028   334.29   336.86  1.0737
      100.0  0.001042   418.24   420.85  1.3050
      140.0  0.001078   587.82   590.52  1.7369
...               ...      ...      ...     ...
300.0 20.0   0.000989    82.17   111.84  0.2899
      40.0   0.000995   164.04   193.89  0.5607
      100.0  0.001029   410.78   441.66  1.2844
      200.0  0.001130   831.40   865.30  2.2893
      300.0  0.001330  1287.90  1327.80  3.1741

[66 rows x 4 columns]

In [112]:
SupWat2

v        u       h       s
Press bar Temp C                                      
0.1       45.806    14.670000  2437.20  2583.9  8.1488
          50.000    14.867000  2443.33  2592.0  8.1741
          55.000    15.101000  2450.59  2601.6  8.2036
          60.000    15.335000  2457.85  2611.2  8.2326
          65.000    15.568000  2465.02  2620.7  8.2611
...                       ...      ...     ...     ...
10000.0   1200.000   0.001343  3674.00  5016.7  5.7036
          1400.000   0.001452  4244.80  5697.2  6.1368
          1600.000   0.001563  4812.00  6374.7  6.5193
          1800.000   0.001672  5376.70  7048.2  6.8609
          2000.000   0.001778  5939.80  7717.5  7.1691

[6956 rows x 4 columns]

# Superheated Water Vapor Calculation

In [117]:
class NonSat:
    def __init__(self, df):
        self.df = df
        self.pressure_list = np.unique(df.index.get_level_values(0))

    def where_pressure(self, p):
        """Determines whether pressure is on table."""
        if p in self.pressure_list:
            return self.pressure_list.tolist().index(p)  # returns index (int) of pressure table
        elif p < self.pressure_list[0] or p > self.pressure_list[-1]:
            return f'Pressure must be between {self.pressure_list[0]} - {self.pressure_list[-1]} bar.'  # pressure not in pressure table
        elif self.pressure_list[0] < p < self.pressure_list[-1]:
            pres_position = np.searchsorted(self.pressure_list, p)
            return pres_position  # returns where pressure is between table

    def interpolation_instruction(self, p):
        """Gives instructions on what path to follow based on where pressure is on table."""
        if type(self.where_pressure(p)) is int:
            return 'on table'
        elif type(self.where_pressure(p)) is np.int64:
            return 'between table'
        else:
            return self.where_pressure(p)

    def interpolate_temp(self, p, t):
        """Simple interpolation given a pressure on the table and a random temperature.
            Returns dictionary with all elements from table."""
        temp_df = self.df.xs(p, level=0).T
        position = np.searchsorted(temp_df.keys(), t)
        x = (t - temp_df.keys()[position - 1]) / (temp_df.keys()[position] - temp_df.keys()[position - 1])
        list_df = self.df.xs(p, level=0)
        dictionary = {}
        for names in self.df.xs(p, level=0).columns:
            y = x * (list_df[names].values[position] - list_df[names].values[position - 1]) + list_df[names].values[
                position - 1]
            dictionary.update({names: y})
        return dictionary

    def interpolate_temp_from_pressure(self, p, z, column):
        """Simple interpolation given a pressure on the table and a random element.
            Returns temperature for the corresponding element."""
        list_df = self.df.xs(p, level=0)[column]
        position = np.searchsorted(list_df.values, z)
        x = (z - list_df.values[position - 1]) / (list_df.values[position] - list_df.values[position - 1])
        y = x * (list_df.keys()[position] - list_df.keys()[position - 1]) + list_df.keys()[position - 1]
        return y

    def flow_to_interpolate_from_pressure(self, p, z, column):
        """Gives instructions for how to execute code given a pressure on the table and a random element."""
        if z > self.df.xs(p, level=0)[column].values[-1]:
            return f'Inputted value ({z}) too large'
        elif z < self.df.xs(p, level=0)[column].values[0]:
            return f'Inputted value ({z}) too small'
        elif z == self.df.xs(p, level=0)[column].values[-1]:
            return self.df.xs(p, level=0)[column].keys()[-1]
        elif z == self.df.xs(p, level=0)[column].values[0]:
            return self.df.xs(p, level=0)[column].keys()[0]
        else:
            return self.interpolate_temp_from_pressure(p, z, column)

    def pretty_return(self, p, z, column):
        """Code wrapper for efficient output."""
        output = self.flow_to_interpolate_from_pressure(p, z, column)
        if type(output) is np.float64:
            return [output, self.interpolate_temp(p, output)]
        else:
            return output

    @staticmethod
    def output_final_dict(dh, dl, xp):
        """Returns final interpolated values for each element in table using pressure position."""
        fd = {}
        for k in dh:
            z = xp * (dh.get(k) - dl.get(k)) + dl.get(k)
            fd.update({k: z})
        return fd

    def triple_interpolation_variables(self, p):
        """Establishes variables used in triple interpolation."""
        pres_position = np.searchsorted(self.pressure_list, p)
        x_pres = (p - self.pressure_list[pres_position - 1]) / (self.pressure_list[pres_position] - self.pressure_list[pres_position - 1])
        temp_df_high = self.df.xs(self.pressure_list[pres_position], level=0).T
        temp_df_low = self.df.xs(self.pressure_list[pres_position - 1], level=0).T
        return [pres_position, x_pres, temp_df_high, temp_df_low]

    def triple_interpolation_t_element(self, section, pres_position, t):
        """Element of triple interpolation when temperature is given.
            Returns dictionary with each interpolated element using temperature position."""
        column_names = self.df.xs(self.pressure_list[pres_position], level=0).columns
        interp_dictionary = {}
        t_pos = np.searchsorted(section.keys(), t)
        x = (t - section.keys()[t_pos - 1]) / (section.keys()[t_pos] - section.keys()[t_pos - 1])
        for names in column_names:
            y = x * (section.T[names].values[t_pos] - section.T[names].values[t_pos - 1]) + section.T[names].values[
                t_pos - 1]
            interp_dictionary.update({names: y})
        return interp_dictionary

    def triple_interpolation_t(self, p, t):
        """Triple interpolation given temperature flow code.
            Returns dictionary of properly interpolated elements."""
        [pres_position, x_pres, temp_df_high, temp_df_low] = self.triple_interpolation_variables(p)

        if temp_df_high.keys()[0] < t < temp_df_high.keys()[-1] and temp_df_low.keys()[0] < t < temp_df_low.keys()[-1]:
            if t in temp_df_high.keys() and t in temp_df_low.keys():
                d_high = temp_df_high[t].to_dict()
                d_low = temp_df_low[t].to_dict()
                return self.output_final_dict(d_high, d_low, x_pres)

            elif t in temp_df_high.keys() and t not in temp_df_low.keys():
                d_high = temp_df_high[t].to_dict()
                d_low = self.triple_interpolation_t_element(temp_df_low, pres_position, t)
                return self.output_final_dict(d_high, d_low, x_pres)

            elif t not in temp_df_high.keys() and t in temp_df_low.keys():
                d_high = self.triple_interpolation_t_element(temp_df_high, pres_position, t)
                d_low = temp_df_low[t].to_dict()
                return self.output_final_dict(d_high, d_low, x_pres)

            else:
                d_high = self.triple_interpolation_t_element(temp_df_high, pres_position, t)
                d_low = self.triple_interpolation_t_element(temp_df_low, pres_position, t)
                return self.output_final_dict(d_high, d_low, x_pres)

        elif t < temp_df_high.keys()[0] or t < temp_df_low.keys()[0]:
            return f'Temperature must be higher than {max([temp_df_high.keys()[0], temp_df_low.keys()[0]])} C'

        elif t > temp_df_high.keys()[-1] or t > temp_df_low.keys()[-1]:
            return f'Temperature must be lower than {min([temp_df_high.keys()[-1], temp_df_low.keys()[-1]])} C'

        elif t == max([temp_df_high.keys()[0], temp_df_low.keys()[0]]):
            max_t_pos = np.argmax([temp_df_high.keys()[0], temp_df_low.keys()[0]])
            if max_t_pos == 0:
                d_high = temp_df_high[t].to_dict()
                d_low = self.triple_interpolation_t_element(temp_df_low, pres_position, t)
                return self.output_final_dict(d_high, d_low, x_pres)

            elif max_t_pos == 1:
                d_high = self.triple_interpolation_t_element(temp_df_high, pres_position, t)
                d_low = temp_df_low[t].to_dict()
                return self.output_final_dict(d_high, d_low, x_pres)

        elif t == min([temp_df_high.keys()[-1], temp_df_low.keys()[-1]]):
            min_t_pos = np.argmin([temp_df_high.keys()[-1], temp_df_low.keys()[-1]])
            if min_t_pos == 0:
                d_high = temp_df_high[t].to_dict()
                d_low = self.triple_interpolation_t_element(temp_df_low, pres_position, t)
                return self.output_final_dict(d_high, d_low, x_pres)

            elif min_t_pos == 1:
                d_high = self.triple_interpolation_t_element(temp_df_high, pres_position, t)
                d_low = temp_df_low[t].to_dict()
                return self.output_final_dict(d_high, d_low, x_pres)

    def triple_interpolation_z_element(self, temp_df, zdf, z, pres_position, inout):
        """Element of triple interpolation when an elemental value is given.
            Depending on whether the elemental value appears on a table, it follows a flow where both instances
            return its corresponding temperature and
            dictionary with each interpolated element."""
        if inout == 'in':
            z_pos = np.where(zdf.values == z)[0][0]
            y = temp_df.keys()[z_pos]
            return [y, temp_df[y].to_dict()]
        elif inout == 'out':
            z_pos = np.searchsorted(zdf.values, z)
            x = (z - zdf.values[z_pos - 1]) / (zdf.values[z_pos] - zdf.values[z_pos - 1])
            y = x * (zdf.keys()[z_pos] - zdf.keys()[z_pos - 1]) + zdf.keys()[z_pos - 1]
            return [y, self.triple_interpolation_t_element(temp_df, pres_position, y)]

    def triple_interpolation_z(self, p, z, column):
        """Triple interpolation given an elemental value; flow code.
            Returns list with interpolated temperature and dictionary of properly interpolated elements."""
        [pres_position, x_pres, temp_df_high, temp_df_low] = self.triple_interpolation_variables(p)
        zdf_high = self.df.xs(self.pressure_list[pres_position], level=0)[column]
        zdf_low = self.df.xs(self.pressure_list[pres_position - 1], level=0)[column]

        column_units = {'v': 'm3/kg', 'u': 'kJ/kg', 'h': 'kJ/kg', 's': 'kJ/kg-K'}

        if zdf_high.values[0] < z < zdf_high.values[-1] and zdf_low.values[0] < z < zdf_low.values[-1]:

            if z in zdf_high.values and z in zdf_low.values:
                [y_high, d_high] = self.triple_interpolation_z_element(temp_df_high, zdf_high, z, pres_position, 'in')
                [y_low, d_low] = self.triple_interpolation_z_element(temp_df_low, zdf_low, z, pres_position, 'in')
                final_t = x_pres * (y_high - y_low) + y_low
                return [final_t, self.output_final_dict(d_high, d_low, x_pres)]

            elif z in zdf_high.values and z not in zdf_low.values:
                [y_high, d_high] = self.triple_interpolation_z_element(temp_df_high, zdf_high, z, pres_position, 'in')
                [y_low, d_low] = self.triple_interpolation_z_element(temp_df_low, zdf_low, z, pres_position, 'out')
                final_t = x_pres * (y_high - y_low) + y_low
                return [final_t, self.output_final_dict(d_high, d_low, x_pres)]

            elif z not in zdf_high.values and z in zdf_low.values:
                [y_high, d_high] = self.triple_interpolation_z_element(temp_df_high, zdf_high, z, pres_position, 'out')
                [y_low, d_low] = self.triple_interpolation_z_element(temp_df_low, zdf_low, z, pres_position, 'in')
                final_t = x_pres * (y_high - y_low) + y_low
                return [final_t, self.output_final_dict(d_high, d_low, x_pres)]

            else:  # z not in either
                [y_high, d_high] = self.triple_interpolation_z_element(temp_df_high, zdf_high, z, pres_position, 'out')
                [y_low, d_low] = self.triple_interpolation_z_element(temp_df_low, zdf_low, z, pres_position, 'out')
                final_t = x_pres * (y_high - y_low) + y_low
                return [final_t, self.output_final_dict(d_high, d_low, x_pres)]

        elif z < zdf_high.values[0] or z < zdf_low.values[0]:
            return f'Inputted value ({column}) must be higher than {max([zdf_high.values[0], zdf_low.values[0]])} {column_units[column]}'

        elif z > zdf_high.values[-1] or z > zdf_low.values[-1]:
            return f'Inputted value ({column}) must be lower than {min([zdf_high.values[-1], zdf_low.values[-1]])} {column_units[column]}'

        elif z == max([zdf_high.values[0], zdf_low.values[0]]):
            max_z_pos = np.argmax([zdf_high.values[0], zdf_low.values[0]])
            if max_z_pos == 0:
                y_high = temp_df_high.keys()[0]
                d_high = temp_df_high[y_high].to_dict()
                [y_low, d_low] = self.triple_interpolation_z_element(temp_df_low, zdf_low, z, pres_position, 'out')
                final_t = x_pres * (y_high - y_low) + y_low
                return [final_t, self.output_final_dict(d_high, d_low, x_pres)]

            elif max_z_pos == 1:
                [y_high, d_high] = self.triple_interpolation_z_element(temp_df_high, zdf_high, z, pres_position, 'out')
                y_low = temp_df_low.keys()[0]
                d_low = temp_df_low[y_low].to_dict()
                final_t = x_pres * (y_high - y_low) + y_low
                return [final_t, self.output_final_dict(d_high, d_low, x_pres)]

        elif z == min([zdf_high.values[-1], zdf_low.values[-1]]):
            min_z_pos = np.argmin([zdf_high.values[-1], zdf_low.values[-1]])
            if min_z_pos == 0:
                y_high = temp_df_high.keys()[-1]
                d_high = temp_df_high[y_high].to_dict()
                [y_low, d_low] = self.triple_interpolation_z_element(temp_df_low, zdf_low, z, pres_position, 'out')
                final_t = x_pres * (y_high - y_low) + y_low
                return [final_t, self.output_final_dict(d_high, d_low, x_pres)]

            if min_z_pos == 1:
                [y_high, d_high] = self.triple_interpolation_z_element(temp_df_high, zdf_high, z, pres_position, 'out')
                y_low = temp_df_low.keys()[-1]
                d_low = temp_df_low[y_low].to_dict()
                final_t = x_pres * (y_high - y_low) + y_low
                return [final_t, self.output_final_dict(d_high, d_low, x_pres)]

    def outer_dome(self, p=None, t=None, v=None, u=None, h=None, s=None):
        """Flow instructions for reading a compressed liquid / superheated water vapor table."""
        if [p, t, v, u, h, s].count(None) != 4:
            return 'Enter exactly 2 values'

        elif p is None:  # maybe temporary?
            return 'Need a pressure'

        elif p is not None:
            if self.interpolation_instruction(p) == 'on table':
                if t is not None:
                    if self.df.xs(p, level=0).T.keys().max() >= t >= self.df.xs(p, level=0).T.keys().min():
                        if t in self.df.xs(p, level=0).T:
                            return dict(self.df.xs((p, t)))
                        else:
                            return self.interpolate_temp(p, t)
                    else:
                        return f'Temperature must be within the range of {self.df.xs(p, level=0).T.keys().min()} - {self.df.xs(p, level=0).T.keys().max()} C'

                elif v is not None:
                    return self.pretty_return(p, v, 'v')

                elif u is not None:
                    return self.pretty_return(p, u, 'u')

                elif h is not None:
                    return self.pretty_return(p, h, 'h')

                elif s is not None:
                    return self.pretty_return(p, s, 's')

            elif self.interpolation_instruction(p) == 'Pressure not in range':
                return self.interpolation_instruction(p)

            elif self.interpolation_instruction(p) == 'between table':
                if t is not None:
                    return self.triple_interpolation_t(p, t)

                if v is not None:
                    return self.triple_interpolation_z(p, v, 'v')

                if u is not None:
                    return self.triple_interpolation_z(p, u, 'u')

                if h is not None:
                    return self.triple_interpolation_z(p, h, 'h')

                if s is not None:
                    return self.triple_interpolation_z(p, s, 's')

NonSat(SupWat).outer_dome(p=10, v=0.2)


[189.6086206896552,
 {'v': 0.2,
  'u': 2602.089655172414,
  'h': 2802.1413793103447,
  's': 6.638396551724138}]

# Saturated Water Vapor Table Calculation

In [118]:
def return_row(element, column):
    pos = np.where(column == element)[0][0]
    el_dict = SatWat.xs(SatWat.index[pos]).to_dict()
    f_dict = {}
    names = SatWat.index.names
    f_dict.update({names[0]: SatWat.index[pos][0], names[1]: SatWat.index[pos][1]})
    f_dict.update(el_dict)
    return f_dict


class InterpolateRow:
    def __init__(self, ascending_lst_values, ind, element):
        self.lst = ascending_lst_values
        self.ind = ind
        self.elem = element
        self.z_pos = np.searchsorted(self.lst, self.elem)
        self.x = (self.elem - self.lst[self.z_pos - 1]) / (self.lst[self.z_pos] - self.lst[self.z_pos - 1])
        self.t_val = (self.x * (
                self.ind.get_level_values(0)[self.z_pos] - self.ind.get_level_values(0)[self.z_pos - 1]) +
                      self.ind.get_level_values(0)[self.z_pos - 1])
        self.p_val = (self.x * (
                self.ind.get_level_values(1)[self.z_pos] - self.ind.get_level_values(1)[self.z_pos - 1]) +
                      self.ind.get_level_values(1)[self.z_pos - 1])
        self.in_dict = {SatWat.index.names[0]: self.t_val, SatWat.index.names[1]: self.p_val}
        self.empty_dict = {}
        self.names = SatWat.columns

    def simple(self):
        for col in self.names:
            el_val = self.x * (SatWat[col].values[self.z_pos] - SatWat[col].values[self.z_pos - 1]) + \
                     SatWat[col].values[self.z_pos - 1]
            self.empty_dict.update({col: el_val})
        self.in_dict.update(self.empty_dict)
        return self.in_dict

    def complx(self):
        for col in self.names:
            if col == 'Sat Vap ug':
                sw_rev = SatWat['Sat Vap ug'].values[::-1]
                el_val = self.x * (sw_rev[self.z_pos] - sw_rev[self.z_pos - 1]) + sw_rev[self.z_pos - 1]
                self.empty_dict.update({col: el_val})
            elif col == 'Sat Vap hg':
                sw_rev = SatWat['Sat Vap hg'].values[::-1]
                el_val = self.x * (sw_rev[self.z_pos] - sw_rev[self.z_pos - 1]) + sw_rev[self.z_pos - 1]
                self.empty_dict.update({col: el_val})
            elif SatWat[col].values[0] < SatWat[col].values[-1]:  # ascending must be reversed
                el_val = self.x * (SatWat[col].sort_values(ascending=False).values[self.z_pos] -
                                   SatWat[col].sort_values(ascending=False).values[self.z_pos - 1]) + \
                         SatWat[col].sort_values(ascending=False).values[self.z_pos - 1]
                self.empty_dict.update({col: el_val})
            elif SatWat[col].values[0] > SatWat[col].values[-1]:
                el_val = self.x * (SatWat[col].sort_values(ascending=True).values[self.z_pos] -
                                   SatWat[col].sort_values(ascending=True).values[self.z_pos - 1]) + \
                         SatWat[col].sort_values(ascending=True).values[self.z_pos - 1]
                self.empty_dict.update({col: el_val})
        self.in_dict.update(self.empty_dict)
        return self.in_dict


class QualityCalculation:
    @staticmethod
    def non_pt(z, column):
        if SatWat[column].values[0] < SatWat[column].values[-1]:  # already ascending: only happens with f columns
            sw_ascend = SatWat[column].sort_values().values
            sw_index = SatWat[column].index
            if pd.Series.between(z, sw_ascend[0], sw_ascend[-1]):
                if z in SatWat[column].values:
                    return return_row(z, SatWat[column].values)
                else:
                    return InterpolateRow(sw_ascend, sw_index, z).simple()
            else:
                f'{z} is not in range for {column}.\nMust be within range: {SatWat[column].values[0]}-{SatWat[column].values[-1]}.'

        elif SatWat[column].values[0] > SatWat[column].values[-1]:  # descending: only happens with vg and sg
            sw_ascend = SatWat[column].sort_values().values
            sw_index = SatWat[column].sort_values().index
            if pd.Series.between(z, sw_ascend[0], sw_ascend[-1]):
                if z in SatWat[column].values:
                    return return_row(z, SatWat[column].values)
                else:
                    return InterpolateRow(sw_ascend, sw_index, z).complx()
            else:
                f'{z} is not in range for {column}.\nMust be within range: {SatWat[column].values[-1]}-{SatWat[column].values[0]}.'

    @staticmethod
    def d_pop(dictionary, line):
        for x in list(dictionary.keys()):
            if f'{line}' in x:
                dictionary.pop(x)
            else:
                pass
        return dictionary

    @staticmethod
    def values(dictionary, q):
        new_labels = [list(dictionary.keys())[0], list(dictionary.keys())[1], 'v', 'u', 'h', 's', 'quality']
        d_values = list(dictionary.values())
        v_val = q * (d_values[3] - d_values[2]) + d_values[2]
        u_val = q * (d_values[5] - d_values[4]) + d_values[4]
        h_val = q * (d_values[7]) + d_values[6]
        s_val = q * (d_values[10] - d_values[9]) + d_values[9]
        new_values = [d_values[0], d_values[1], v_val, u_val, h_val, s_val, q]
        return dict(zip(new_labels, new_values))

    @staticmethod
    def pt(ptx, q, index):
        if q == 0:  # liq line
            if ptx in index:
                d = return_row(ptx, index)
                return QualityCalculation().d_pop(d, 'g')
            elif index[0] < ptx < index[-1]:
                d = InterpolateRow(index, SatWat.index, ptx).simple()
                return QualityCalculation().d_pop(d, 'g')
            else:
                return f'{ptx} value must be within range of {index[0]} - {index[-1]}'
        elif q == 1:  # vap line
            if ptx in index:
                d = return_row(ptx, index)
                return QualityCalculation().d_pop(d, 'f')
            elif index[0] < ptx < index[-1]:
                d = InterpolateRow(index, SatWat.index, ptx).simple()
                return QualityCalculation().d_pop(d, 'f')
            else:
                return f'{ptx} value must be within range of {index[0]} - {index[-1]}'
        else:
            if ptx in index:
                d = return_row(ptx, index)
                return QualityCalculation().values(d, q)
            elif index[0] < ptx < index[-1]:
                d = InterpolateRow(index, SatWat.index, ptx).simple()
                return QualityCalculation().values(d, q)
            else:
                return f'{ptx} value must be within range of {index[0]} - {index[-1]}'


def saturated(p=None, t=None, v=None, u=None, h=None, s=None, quality=None):
    if [p, t].count(None) == 2:  # expect 1 of v/u/h/s and quality of 0 or 1
        if [v, u, h, s].count(None) != 3:
            return 'Without a pressure or temperature,\nonly enter 1 value out of v, u, h, and s WITH a quality of 0 or 1.\n(signifies value on the saturated liq/vap line)'
        else:
            if quality == 1:  # interpolate using vapor line
                if v is not None:
                    return QualityCalculation().non_pt(v, 'Sat Vap vg')
                elif u is not None:
                    return 'Cannot interpolate off of vapor line only using internal energy and quality.'
                elif h is not None:
                    return 'Cannot interpolate off of vapor line only using enthalpy and quality.'
                elif s is not None:
                    return QualityCalculation().non_pt(s, 'Sat Vap sg')

            elif quality == 0:
                if v is not None:
                    return QualityCalculation().non_pt(v, 'Sat Liq vf')
                elif u is not None:
                    return QualityCalculation().non_pt(u, 'Sat Liq uf')
                elif h is not None:
                    return QualityCalculation().non_pt(h, 'Sat Liq hf')
                elif s is not None:
                    return QualityCalculation().non_pt(s, 'Sat Liq sf')

            elif 0 < quality < 1:
                return 'Without a pressure or temperature,\nonly enter 1 value out of v, u, h, and s WITH a quality of 0 or 1.\n(signifies value on the saturated liq/vap line)'
            else:
                return 'Without a pressure or temperature,\nonly enter 1 value out of v, u, h, and s WITH a quality of 0 or 1.\n(signifies value on the saturated liq/vap line)'

    elif [p, t].count(None) == 1:
        if p is not None:
            if quality is not None:  # p and q
                if 0 <= quality <= 1:
                    if [v, u, h, s].count(None) == 4:
                        return QualityCalculation().pt(p, quality, SatWat.index.get_level_values(1))
                    else:
                        return 'Given a pressure, including quality and another value is redundant.\nFor this, choose one.'
                else:
                    return 'Quality must be between 0 and 1.'

            elif quality is None:
                if [v, u, h, s].count(None) == 4:  # only p
                    if p in SatWat.index.get_level_values(1):
                        return return_row(p, SatWat.index.get_level_values(1))
                    elif SatWat.index.get_level_values(1)[0] < p < SatWat.index.get_level_values(1)[-1]:
                        return InterpolateRow(SatWat.index.get_level_values(1), SatWat.index, p).simple()
                    else:
                        return 'Pressure not in saturated range'

                elif [v, u, h, s].count(None) == 3:  # p and v/u/h/s
                    if SatWat.index.get_level_values(1)[0] <= p < SatWat.index.get_level_values(1)[-1]:
                        if v is not None:
                            lst = list(saturated(p=p).values())
                            q = (v - lst[2]) / (lst[3] - lst[2])
                            return saturated(p=p, quality=q)
                        elif u is not None:
                            lst = list(saturated(p=p).values())
                            q = (u - lst[4]) / (lst[5] - lst[4])
                            return saturated(p=p, quality=q)
                        elif h is not None:
                            lst = list(saturated(p=p).values())
                            q = (h - lst[6]) / (lst[7])
                            return saturated(p=p, quality=q)
                        elif s is not None:
                            lst = list(saturated(p=p).values())
                            q = (s - lst[9]) / (lst[10] - lst[9])
                            return saturated(p=p, quality=q)

                    elif p == SatWat.index.get_level_values(1)[-1]:
                        return f'At {SatWat.index.get_level_values(1)[-1]} bar,\ngiving an additional property (v/u/h/s) is redundant.\nOnly the pressure is necessary.'
                    else:
                        return 'Pressure not in saturated range.'
                else:
                    return 'Entering a pressure and 2 other properties (v/u/h/s) is redundant.\nEnter a pressure and one of the properties.'

        elif t is not None:
            if quality is not None:  # t and q
                if 0 <= quality <= 1:
                    if [v, u, h, s].count(None) == 4:
                        return QualityCalculation().pt(t, quality, SatWat.index.get_level_values(0))
                    else:
                        return 'Given a temperature, including quality and another value is redundant.\nFor this, choose one.'
                else:
                    return 'Quality must be between 0 and 1.'

            elif quality is None:
                if [v, u, h, s].count(None) == 4:  # only t
                    if t in SatWat.index.get_level_values(0):
                        return return_row(t, SatWat.index.get_level_values(0))
                    elif SatWat.index.get_level_values(0)[0] < t < SatWat.index.get_level_values(0)[-1]:
                        return InterpolateRow(SatWat.index.get_level_values(0), SatWat.index, t).simple()
                    else:
                        return 'Temperature not in saturated range'

                elif [v, u, h, s].count(None) == 3:  # t and v/u/h/s
                    if SatWat.index.get_level_values(0)[0] <= t < SatWat.index.get_level_values(0)[-1]:
                        if v is not None:
                            lst = list(saturated(t=t).values())
                            q = (v - lst[2]) / (lst[3] - lst[2])
                            return saturated(t=t, quality=q)
                        elif u is not None:
                            lst = list(saturated(t=t).values())
                            q = (u - lst[4]) / (lst[5] - lst[4])
                            return saturated(t=t, quality=q)
                        elif h is not None:
                            lst = list(saturated(t=t).values())
                            q = (h - lst[6]) / (lst[7])
                            return saturated(t=t, quality=q)
                        elif s is not None:
                            lst = list(saturated(t=t).values())
                            q = (s - lst[9]) / (lst[10] - lst[9])
                            return saturated(t=t, quality=q)

                    elif t == SatWat.index.get_level_values(0)[-1]:
                        return f'At {SatWat.index.get_level_values(0)[-1]} bar, giving an additional property (v/u/h/s) is redundant.\nOnly the temperature is necessary.'
                    else:
                        return 'Temperature not in saturated range.'
                else:
                    return 'Entering a temperature and 2 other properties (v/u/h/s) is redundant.\nEnter a temperature and one of the properties.'

    elif [p, t].count(None) == 0:
        return 'In a saturated values table,\npressure and temperature are equivalent values (redundant).\nPlease only use one.'

saturated(p=62.368, quality=0.43)


{'Temp': 278.12893203883493,
 'Pres': 62.368,
 'v': 0.014144228413592233,
 'u': 1806.9994932038835,
 'h': 1895.1477242718447,
 's': 4.262778852427184,
 'quality': 0.43}